In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LANGSMITH_API_KEY: \n")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY: \n")

In [3]:
from langchain_tavily import TavilySearch

tavily_api_key = os.environ.get("TAVILY_API_KEY")
if not tavily_api_key:
	raise ValueError("TAVILY_API_KEY environment variable is not set or is empty. Please set it before running this cell.")

search = TavilySearch(max_results=2, tavily_api_key=tavily_api_key)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757238395, 'localtime': '2025-09-07 02:46'}, 'current': {'last_updated_epoch': 1757238300, 'last_updated': '2025-09-07 02:45', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 254, 'wind_dir': 'WSW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 50, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 13.9, 'windchill_f': 57.0, 'heatindex_c': 14.2, 'heatindex_f': 57.6, 'dewpoint_c': 13.7, 